In [ ]:
# user 1 inputs text in gradio
# save user 1 input as variable
# text is passed to a model as input
# returns image
# image is displayed in gradio

# user 2 inputs text in gradio
# save user 2 input as variable
# text is combined with user 1's text and passed to model as input
# returns image
# image is displayed in gradio below user 1's image

In [2]:
import gradio as gr

prompts = []

def add_text(new_text):
    prompts.append(new_text)
    prompt_n = f"prompt_{len(prompts)}"
    combined = " ".join(prompts)
    return f"{prompt_n}: {combined}"

interface = gr.Interface(
    fn=add_text,
    inputs=gr.Textbox(label="Enter text"),
    outputs=gr.Textbox(label="Combined text"),
    title="Text Accumulator",
    description="Enter text and see it accumulate with previous entries"
)

interface.launch()


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [1]:
# !pip install diffusers transformers
from diffusers import DiffusionPipeline
import torch

import gradio as gr

prompts = []

# Load model once and move to MPS
pipe = DiffusionPipeline.from_pretrained("OFA-Sys/small-stable-diffusion-v0")
pipe.to("mps")

def add_text(new_text):
    prompts.append(new_text)
    prompt_n = f"prompt_{len(prompts)}"
    combined = " ".join(prompts)
    
    image = pipe(new_text).images[0]
    
    if len(prompts) > 1:
        combined_image = pipe(combined).images[0]
        return {
            "text": f"{prompt_n}: {combined}",
            "image1": image,
            "image2": combined_image
        }
    
    return {
        "text": f"{prompt_n}: {combined}",
        "image1": image
    }

interface = gr.Interface(
    fn=add_text,
    inputs=gr.Textbox(label="Enter text"),
    outputs=[
        gr.Textbox(label="Combined text"),
        gr.Image(label="Current prompt image"),
        gr.Image(label="Combined prompts image", visible=False)
    ],
    title="Text and Image Generator",
    description="Enter text to generate images from individual and combined prompts"
)

interface.launch()


/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Loading pipeline components...:  29%|██▊       | 2/7 [00:00<00:00, 13.07it/s]An error occurred while trying to fetch /Users/joshstrupp/.cache/huggingface/hub/models--OFA-Sys--small-stable-diffusion-v0/snapshots/38e10e5e71e8fbf717a47a81e7543cd01c1a8140/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /Users/joshstrupp/.cache/huggingface/hub/models--OFA-Sys--small-stable-diffusion-v0/snapshots/38e10e5e71e8fbf717a47a81e7543cd01c1a8140/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'predict_epsilon': True} were passed to DPMSolverMultistepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
An error occurred while trying to fetch /Users/joshstrupp/.cache/huggingface/hub/models--OFA-Sys--small-stable-diffusion-v0/snapshots/38e10e5e71e8fbf717a47a81e7543cd01c1a8140/unet: Error no file named diffusion_pytorch_model.safetensors found i

KeyboardInterrupt: 

In [1]:
import gradio as gr

prompts = []

def add_text(new_text):
    prompts.append(new_text)
    prompt_n = f"prompt_{len(prompts)}"
    combined = " ".join(prompts)
    return f"{prompt_n}: {combined}"

interface = gr.Interface(
    fn=add_text,
    inputs=gr.Textbox(label="Enter text"),
    outputs=gr.Textbox(label="Combined text"),
    title="Text Accumulator",
    description="Enter text and see it accumulate with previous entries"
)

interface.launch()


/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [8]:
%pip install openai
import gradio as gr
from openai import OpenAI
import requests
from PIL import Image
from io import BytesIO
import os

prompts = []

def add_text(new_text):
    prompts.append(new_text)
    prompt_n = f"prompt_{len(prompts)}"
    combined = " ".join(prompts)
    
    # Generate image using DALL-E
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("Please set OPENAI_API_KEY in your .env file")
    client = OpenAI(api_key=api_key)
    
    # Generate image for new text
    response1 = client.images.generate(
        model="dall-e-2",
        prompt=new_text,
        size="512x512",
        quality="standard",
        n=1,
    )
    
    # Get image URL and download image for new text
    image_url1 = response1.data[0].url
    response_img1 = requests.get(image_url1)
    image1 = Image.open(BytesIO(response_img1.content))
    
    # Generate image for combined text
    response2 = client.images.generate(
        model="dall-e-2",
        prompt=combined,
        size="512x512",
        quality="standard",
        n=1,
    )
    
    # Get image URL and download image for combined text
    image_url2 = response2.data[0].url
    response_img2 = requests.get(image_url2)
    image2 = Image.open(BytesIO(response_img2.content))
    
    # Return values directly matching the output components
    return f"{prompt_n}: {combined}", image1, image2

interface = gr.Interface(
    fn=add_text,
    inputs=gr.Textbox(label="Enter text"),
    outputs=[
        gr.Textbox(label="Combined text"),
        gr.Image(label="Current prompt image"),
        gr.Image(label="Combined prompts image")
    ],
    title="Text and Image Generator", 
    description="Enter text to generate images using DALL-E"
)
interface.launch()


Note: you may need to restart the kernel to use updated packages.
Running on local URL:  http://127.0.0.1:7872

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.
